In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from biot_savart_v4_3 import parse_coil, plot_coil, slice_coil

In [ ]:
# COIL = '6'
COIL = "12"

# load up the simple spiral coil
coil1 = parse_coil(f"coils/coil_{COIL}_spiral.csv")
plot_coil(f"coils/coil_{COIL}_spiral.csv")
coil1 = slice_coil(coil1, 0.01)
coil1 = coil1.T
print(coil1.shape)

coil2 = parse_coil(f"coils/coil_{COIL}_custom.csv")
plot_coil(f"coils/coil_{COIL}_custom.csv")
coil2 = slice_coil(coil2, 0.01)
coil2 = coil2.T
print(coil2.shape)

In [ ]:
# show a simple dipole magnet
# magnetic field at a point x,y,z of a dipole magnet with moment m in the z direction
def dipole(x, y, z, m=0.185):
    mu0 = 1e-7 / 4 * np.pi
    r = np.sqrt(x ** 2 + y ** 2 + z ** 2)
    return (
        np.array(
            [3 * x * z / r ** 5, 3 * y * z / r ** 5, (3 * z ** 2 / r ** 5 - 1 / r ** 3)]
        )
        * m
        * mu0
    )


def plot_field_slice(x, y, bx, by, mag, name="magnetic_field.png", draw_magnet=True):
    # plot the magnetic field
    fig = plt.figure()
    ax = fig.add_subplot(111)
    ax.streamplot(
        x,
        y,
        bx,
        by,
        linewidth=1,
        cmap=plt.cm.inferno,
        density=2,
        arrowstyle="->",
        arrowsize=1.5,
    )

    ax.set_xlabel("$x$")
    ax.set_ylabel("$y$")
    ax.set_xlim(-0.1, 0.1)
    ax.set_ylim(-0.1, 0.1)
    ax.set_aspect("equal")

    # plot the magniture of the field as an image
    im = ax.imshow(
        mag, extent=[-0.1, 0.1, -0.1, 0.1], origin="lower", cmap=plt.cm.inferno
    )
    if draw_magnet:
        # draw the magnet
        ax.add_patch(
            plt.Rectangle((-0.005, -0.0015), 0.01, 0.003, fc="w", ec="k", lw=1)
        )

    # make the figure bigger
    fig.set_size_inches(10, 10)

    fig.show()
    # save the figure
    fig.savefig(name)


# # calculate the magnetic field at y = 0, over z = -1, 1 and x = -1, 1
x = np.linspace(-0.1, 0.1, 100)
z = np.linspace(-0.1, 0.1, 100)
X, Z = np.meshgrid(x, z)
Bx, By, Bz = dipole(X, 0, Z)

print(Bx.shape, By.shape, Bz.shape)

plot_field_slice(
    X,
    Z,
    Bx,
    Bz,
    np.log(np.sqrt(Bx ** 2 + By ** 2 + Bz ** 2)),
    "dipole_field_side.png",
    False,
)

# # calculate the magnetic field at z = 1, over y = -1, 1 and x = -1, 1
x = np.linspace(-0.1, 0.1, 100)
y = np.linspace(-0.1, 0.1, 100)
X, Y = np.meshgrid(x, y)
Bx, By, Bz = dipole(X, Y, 0.01)

plot_field_slice(
    X,
    Y,
    Bx,
    By,
    np.log(np.sqrt(Bx ** 2 + By ** 2 + Bz ** 2)),
    "dipole_field_bottom.png",
    False,
)

# Simple Simulation of a dipole magnet

In [ ]:
def B(x, y, z, m=0.185, l=0.003, d=0.01):
    d = d * 0.75
    l = l * 0.75
    # simulate multiple points in the cylinder and add them together to create a disk of field
    bx = 0
    by = 0
    bz = 0
    for degrees in range(0, 360, 30):
        angle = np.deg2rad(degrees)
        x_, y_, z_ = dipole(
            x + d / 2 * np.cos(angle),
            y + d / 2 * np.sin(angle),
            z - l / 2,
            m / (360 / 30),
        )
        bx += x_
        by += y_
        bz += z_
        x_, y_, z_ = dipole(
            x + d / 2 * np.cos(angle),
            y + d / 2 * np.sin(angle),
            z + l / 2,
            m / (360 / 30),
        )
        bx += x_
        by += y_
        bz += z_
    return np.array([bx, by, bz])


# # calculate the magnetic field at y = 0, over z = -1, 1 and x = -1, 1
x = np.linspace(-0.1, 0.1, 100)
z = np.linspace(-0.1, 0.1, 100)
X, Z = np.meshgrid(x, z)
Bx, By, Bz = B(X, 0, Z)

print(Bx.shape, By.shape, Bz.shape)

plot_field_slice(
    X,
    Z,
    Bx,
    Bz,
    np.log(np.sqrt(Bx ** 2 + By ** 2 + Bz ** 2)),
    "magnetic_field_side.png",
)

# # calculate the magnetic field at z = 1, over y = -1, 1 and x = -1, 1
x = np.linspace(-0.1, 0.1, 100)
y = np.linspace(-0.1, 0.1, 100)
X, Y = np.meshgrid(x, y)
Bx, By, Bz = B(X, Y, 0.01)

plot_field_slice(
    X,
    Y,
    Bx,
    By,
    np.log(np.sqrt(Bx ** 2 + By ** 2 + Bz ** 2)),
    "magnetic_field_bottom.png",
)

# calculate the magnetic field in a 3d volume
x = np.linspace(-0.1, 0.1, 100)
y = np.linspace(-0.1, 0.1, 100)
z = np.linspace(-0.1, 0.1, 100)
X, Y, Z = np.meshgrid(x, y, z)
Bx, By, Bz = B(X, Y, Z)

In [ ]:
# do a 3d quivwer plot of the magnetic field
fig = plt.figure()
ax = fig.add_subplot(111, projection="3d")
# down sample the results
ax.quiver(
    X[::10, ::10, ::10],
    Y[::10, ::10, ::10],
    Z[::10, ::10, ::10],
    Bx[::10, ::10, ::10],
    By[::10, ::10, ::10],
    Bz[::10, ::10, ::10],
    length=0.01,
    normalize=True,
)
ax.set_xlabel("$x$")
ax.set_ylabel("$y$")
ax.set_zlabel("$z$")
ax.set_xlim(-0.1, 0.1)
ax.set_ylim(-0.1, 0.1)
ax.set_zlim(-0.1, 0.1)
ax.set_aspect("equal")
# make the plot larger
fig.set_size_inches(10, 10)
fig.show()

In [ ]:
# calculate the force on a wire of length l carrying current I at a point x,y,z with a direction vector d
def F(p, d, I, l):
    return I * l * np.cross(d, B(p[0], p[1], p[2]))


def calculate_forces_on_wire_points(points):
    Fx = []
    Fy = []
    Fz = []
    # calculate the force on each point
    for i in range(len(points) - 1):
        # calculate the direction vector
        dx = points[i][0] - points[(i + 1)][0]
        dy = points[i][1] - points[(i + 1)][1]
        dz = points[i][2] - points[(i + 1)][2]
        d = np.array([dx, dy, dz])
        # get the length of d
        l = np.sqrt(dx ** 2 + dy ** 2 + dz ** 2)
        if l > 0:
            # normalise d
            d = d / l
            # calculate the force
            fx, fy, fz = F(points[i], d, points[i][3], l)
            Fx.append(fx)
            Fy.append(fy)
            Fz.append(fz)
        else:
            Fx.append(0)
            Fy.append(0)
            Fz.append(0)
    return Fx, Fy, Fz

In [ ]:
# instead of sweeping horizontally, we'll sweep the coils around a circle
def sweep_coil_circle(coil, coil_center_radius, theta):
    X = coil_center_radius * np.cos(np.deg2rad(theta))
    Y = coil_center_radius * np.sin(np.deg2rad(theta)) - coil_center_radius
    Z = -0.002

    # loop through the locations and calculate the forces, sum up the force in the X direction for each location
    Fx = []
    Fy = []
    Fz = []
    for p in range(len(theta)):
        x = X[p]
        y = Y[p]
        z = Z

        points = coil.copy()
        for i in range(len(points)):
            points[i][0] = points[i][0] / 100 + x
            points[i][1] = points[i][1] / 100 + y
            points[i][2] = points[i][2] / 100 + z
        # feel the force
        Fx_, Fy_, Fz_ = calculate_forces_on_wire_points(points)
        Fx.append(sum(Fx_))
        Fy.append(sum(Fy_))
        Fz.append(sum(Fz_))
    return Fx, Fy, Fz


# sweep the coils from -45 to 45 degrees in 1 degree steps
theta = np.linspace(25, 175, 100)

# do a quick sanity check
# coil center in meters
coil_center_radius = 20.5 / 1000
X = coil_center_radius * np.cos(np.deg2rad(theta))
Y = coil_center_radius * np.sin(np.deg2rad(theta)) - coil_center_radius

# plot X and Y along with the coil
plt.plot(X, Y, color="red")
points = coil2.copy()
plt.plot(
    # points in meters (coil is in cm)
    [x / 100 for x, y, z, _ in points],
    [y / 100 for x, y, z, _ in points],
    label="coil2",
    color="blue",
)
points = coil1.copy()
plt.plot(
    # points in meters (coil is in cm)
    [x / 100 for x, y, z, _ in points],
    [y / 100 for x, y, z, _ in points],
    label="coil1",
    color="red",
)
# make the aspect ratio equal
plt.gca().set_aspect("equal")

Fx_1_curve, Fy_1_curve, Fz_1_curve = sweep_coil_circle(coil1, 20.5 / 1000, theta)
Fx_2_curve, Fy_2_curve, Fz_2_curve = sweep_coil_circle(coil2, 20.5 / 1000, theta)

In [ ]:
plt.plot(theta, np.array(Fx_1_curve), label="coil1", color="red")
plt.plot(theta, -np.array(Fx_2_curve), label="coil2", color="blue")
# plot a dotted line along y = 0
plt.gcf().set_size_inches(16, 9)
plt.plot([theta[0], theta[-1]], [0, 0], "--")
print(np.max(np.abs(Fx_1_curve)))
# save the plot
plt.savefig(f"torque_{COIL}_coil_sweep_circle_1mm_spiral.png")